# Title: Robust_Label_Encoder_for_Category
## Description: 
Train 셋에 존재하지 않는 Test 셋의 고유 라벨(Unseen Label)을 식별하여 인코더 클래스에 동적으로 추가함으로써, 인코딩 에러를 방지하고 일관된 변환을 수행함.
## Input: 
- `Train DataFrame`
- `Test DataFrame`
- `List of Categorical Column Names`
## Output: 
- `Label Encoded Train DataFrame` 
- `Label Encoded Test DataFrame`
## Check Point: 
- Data Leakage 주의 (대회용으로는 적합하나, 엄격한 실시간 추론 환경에서는 Unknown 토큰 처리가 더 나을 수 있음)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def robust_label_encoding(train_df, test_df, cat_cols):
    """
    Train 데이터에 핏(fit)한 후, Test 데이터에만 있는 라벨을 감지하여
    LabelEncoder의 클래스를 확장하고 변환을 수행하는 함수
    """
    # 원본 데이터 보존을 위해 복사
    train_encoded = train_df.copy()
    test_encoded = test_df.copy()

    for col in cat_cols:
        le = LabelEncoder()
        le.fit(train_encoded[col])

        # Test 데이터의 고유값 확인
        for label in np.unique(test_encoded[col]):
            if label not in le.classes_:
                # Train에 없는 라벨이 Test에 있다면 클래스에 추가
                le.classes_ = np.append(le.classes_, label)

        # 변환 수행
        train_encoded[col] = le.transform(train_encoded[col])
        test_encoded[col] = le.transform(test_encoded[col])

    return train_encoded, test_encoded

## How to Use
- `cat_cols` 리스트에 인코딩할 범주형 변수명(String)을 담아 함수에 전달합니다.
- `train_encoded, test_encoded = robust_label_encoding(train, test, str_col)` 형태로 호출하여 사용합니다.

## Troubleshooting
- **TypeError**: `cat_cols`에 수치형 변수(float 등)가 포함되어 있고 NaN 값이 있을 경우 발생할 수 있습니다. 사전에 결측치를 처리하거나 string으로 형변환(`astype(str)`)이 필요합니다.
- **MemoryError**: 데이터프레임의 크기가 매우 클 경우 `copy()` 수행 시 메모리 부족이 발생할 수 있습니다. 이 경우 함수 내부에서 원본을 직접 수정(`inplace`)하도록 변경하세요.